Notebook này chạy trên local.

# Binary Prediction of Poisonous Mushrooms

[Link_to_competition](https://www.kaggle.com/competitions/playground-series-s4e8)

In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'

In [2]:
!kaggle competitions download -c playground-series-s4e8

playground-series-s4e8.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
from zipfile import ZipFile
with ZipFile('playground-series-s4e8.zip') as f:
    f.extractall('poisonous_mushroom_data')

In [4]:
data_dir = 'poisonous_mushroom_data/'

In [5]:
import pandas as pd

In [6]:
raw_df = pd.read_csv(data_dir + 'train.csv')
test_df = pd.read_csv(data_dir + 'test.csv')
sub_df = pd.read_csv(data_dir + 'sample_submission.csv')

In [7]:
raw_df

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,3116940,e,9.29,f,NaN,n,t,NaN,NaN,w,...,b,NaN,w,u,w,t,g,NaN,d,u
3116941,3116941,e,10.88,s,NaN,w,t,d,c,p,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
3116942,3116942,p,7.82,x,e,e,f,a,NaN,w,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
3116943,3116943,e,9.45,p,i,n,t,e,NaN,p,...,NaN,y,w,NaN,NaN,t,p,NaN,d,u


In [8]:
raw_df.dtypes

id                        int64
class                    object
cap-diameter            float64
cap-shape                object
cap-surface              object
cap-color                object
does-bruise-or-bleed     object
gill-attachment          object
gill-spacing             object
gill-color               object
stem-height             float64
stem-width              float64
stem-root                object
stem-surface             object
stem-color               object
veil-type                object
veil-color               object
has-ring                 object
ring-type                object
spore-print-color        object
habitat                  object
season                   object
dtype: object

In [9]:
raw_df.isna().sum()

id                            0
class                         0
cap-diameter                  4
cap-shape                    40
cap-surface              671023
cap-color                    12
does-bruise-or-bleed          8
gill-attachment          523936
gill-spacing            1258435
gill-color                   57
stem-height                   0
stem-width                    0
stem-root               2757023
stem-surface            1980861
stem-color                   38
veil-type               2957493
veil-color              2740947
has-ring                     24
ring-type                128880
spore-print-color       2849682
habitat                      45
season                        0
dtype: int64

In [10]:
numeric_cols = ['cap-diameter', 'stem-height', 'stem-width']
categorical_cols = raw_df.select_dtypes(include='object').columns.tolist()

In [11]:
categorical_cols.remove('class')

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [13]:
categorical_cols

['cap-shape',
 'cap-surface',
 'cap-color',
 'does-bruise-or-bleed',
 'gill-attachment',
 'gill-spacing',
 'gill-color',
 'stem-root',
 'stem-surface',
 'stem-color',
 'veil-type',
 'veil-color',
 'has-ring',
 'ring-type',
 'spore-print-color',
 'habitat',
 'season']

In [14]:
raw_df['class'].value_counts()

class
p    1705396
e    1411549
Name: count, dtype: int64

In [15]:
def view_categoricals_feature(cat_col: str):
    d = dict()    
    total = raw_df[cat_col].value_counts()
    idx = raw_df['class'].value_counts().index.tolist()
    for t in idx:
        x = raw_df[raw_df['class']==t][cat_col].value_counts()
        d[t] = x/total*100
    
    return pd.DataFrame(data=d, columns=idx)

## Exploring and preprocessing

### Cap

In [16]:
raw_df['cap-shape'].value_counts().head(15)

cap-shape
x    1436026
f     676238
s     365146
b     318646
o     108835
p     106967
c     104520
d         65
e         60
n         41
t         36
w         36
g         34
y         33
r         32
Name: count, dtype: int64

In [17]:
cap_shape_lst = ['x','f','s','b','o','p','c']
raw_df['cap-shape'] = raw_df['cap-shape'].apply(lambda x: x if x in cap_shape_lst else 'x')

In [18]:
view_categoricals_feature('cap-shape')

,p,e
cap-shape,,
b,77.303654,22.696346
c,54.277650,45.722350
f,51.194402,48.805598
o,67.481049,32.518951
p,45.940337,54.059663
s,54.945693,45.054307
x,51.018486,48.981514


In [19]:
test_df['cap-shape'] = test_df['cap-shape'].apply(lambda x: x if x in cap_shape_lst else 'x')

In [20]:
cap_surface_lst = raw_df['cap-surface'].value_counts().head(11).index

In [21]:
raw_df['cap-surface'] = raw_df['cap-surface'].apply(lambda x: x if x in cap_surface_lst else 'rem')

In [22]:
view_categoricals_feature('cap-surface')

,p,e
cap-surface,,
d,58.788292,41.211708
e,47.843157,52.156843
g,59.718499,40.281501
h,47.012937,52.987063
i,82.191467,17.808533
k,90.155577,9.844423
l,42.881027,57.118973
rem,51.392220,48.607780
s,36.934307,63.065693


In [23]:
test_df['cap-surface'] = test_df['cap-surface'].apply(lambda x: x if x in cap_surface_lst else 'rem')

In [24]:
cap_color_lst = raw_df['cap-color'].value_counts().head(12).index
cap_color_lst

Index(['n', 'y', 'w', 'g', 'e', 'o', 'p', 'r', 'u', 'b', 'k', 'l'], dtype='object', name='cap-color')

In [25]:
raw_df['cap-color'] = raw_df['cap-color'].apply(lambda x: x if x in cap_color_lst else 'n')

In [26]:
view_categoricals_feature('cap-color')

,p,e
cap-color,,
b,12.604179,87.395821
e,79.324852,20.675148
g,41.191984,58.808016
k,59.848384,40.151616
l,48.928156,51.071844
n,49.092550,50.907450
o,71.726112,28.273888
p,68.993227,31.006773
r,90.693543,9.306457


In [27]:
test_df['cap-color'] = test_df['cap-color'].apply(lambda x: x if x in cap_color_lst else 'n')

### Bruise

In [28]:
raw_df['does-bruise-or-bleed'].value_counts(dropna=False)

does-bruise-or-bleed
f           2569743
t            547085
w                14
c                11
h                 9
NaN               8
y                 7
a                 7
b                 7
x                 7
s                 6
k                 6
p                 4
e                 4
l                 4
d                 4
g                 3
o                 3
z                 3
n                 2
i                 2
has-ring          1
3.43              1
r                 1
4.42              1
2.9               1
u                 1
Name: count, dtype: int64

In [29]:
bruise_lst = ['f','t']
raw_df['does-bruise-or-bleed'] = raw_df['does-bruise-or-bleed'].apply(lambda x: x if x in bruise_lst else 'f')

In [30]:
view_categoricals_feature('does-bruise-or-bleed')

,p,e
does-bruise-or-bleed,,
f,55.576179,44.423821
t,50.662329,49.337671


In [31]:
test_df['does-bruise-or-bleed'] = test_df['does-bruise-or-bleed'].apply(lambda x: x if x in bruise_lst else 'f')

### Gill

In [32]:
raw_df['gill-attachment'].value_counts(dropna=False).head(10)

gill-attachment
a      646034
d      589236
NaN    523936
x      360878
e      301858
s      295439
p      279110
f      119953
c          74
u          56
Name: count, dtype: int64

In [33]:
gill_attachment_lst = raw_df['gill-attachment'].value_counts().head(7).index
raw_df['gill-attachment'] = raw_df['gill-attachment'].apply(lambda x: x if x in gill_attachment_lst else 'rem')

In [34]:
view_categoricals_feature('gill-attachment')

,p,e
gill-attachment,,
a,64.634369,35.365631
d,57.137717,42.862283
e,34.896541,65.103459
f,51.045826,48.954174
p,29.073484,70.926516
rem,63.891182,36.108818
s,58.915715,41.084285
x,53.845067,46.154933


In [35]:
test_df['gill-attachment'] = test_df['gill-attachment'].apply(lambda x: x if x in gill_attachment_lst else 'rem')

In [36]:
raw_df['gill-spacing'].value_counts(dropna=False).head(10)

gill-spacing
c      1331054
NaN    1258435
d       407932
f       119380
e           24
a           17
s           16
b           12
x            8
t            8
Name: count, dtype: int64

In [37]:
gill_spacing_lst = ['c','d','f']
raw_df['gill-spacing'] = raw_df['gill-spacing'].apply(lambda x: x if x in gill_spacing_lst else 'rem')

In [38]:
view_categoricals_feature('gill-spacing')

,p,e
gill-spacing,,
c,57.244560,42.755440
rem,57.083981,42.916019
d,40.224596,59.775404
f,51.016921,48.983079


In [39]:
test_df['gill-spacing'] = test_df['gill-spacing'].apply(lambda x: x if x in gill_spacing_lst else 'rem')

In [40]:
raw_df['gill-color'].value_counts(dropna=False)

gill-color
w         931538
n         543386
y         469464
p         343626
g         212164
           ...  
does w         1
4              1
18.12          1
0.92           1
8.37           1
Name: count, Length: 64, dtype: int64

In [41]:
gill_color_lst = raw_df['gill-color'].value_counts(dropna=False).head(12).index

In [42]:
raw_df['gill-color'] = raw_df['gill-color'].apply(lambda x: x if x in gill_color_lst else 'w')

In [43]:
view_categoricals_feature('gill-color')

,p,e
gill-color,,
b,28.171101,71.828899
e,69.855657,30.144343
f,50.987518,49.012482
g,48.099583,51.900417
k,57.737751,42.262249
n,71.487856,28.512144
o,51.737855,48.262145
p,59.020854,40.979146
r,71.391264,28.608736


In [44]:
test_df['gill-color'] = test_df['gill-color'].apply(lambda x: x if x in gill_color_lst else 'w')

### Stem

In [45]:
raw_df['stem-root'].value_counts(dropna=False).head(15)

stem-root
NaN    2757023
b       165801
s       116946
r        47803
c        28592
f          597
d           24
y           14
w           12
p           12
g           12
k           11
l           10
n           10
t           10
Name: count, dtype: int64

In [46]:
stem_root_lst = ['b','s','r','c']
raw_df['stem-root'] = raw_df['stem-root'].apply(lambda x: x if x in stem_root_lst else 'rem')

In [47]:
view_categoricals_feature('stem-root')

,p,e
stem-root,,
b,35.187363,64.812637
c,99.594292,0.405708
r,99.523042,0.476958
rem,54.003096,45.996904
s,69.865579,30.134421


In [48]:
test_df['stem-root'] = test_df['stem-root'].apply(lambda x: x if x in stem_root_lst else 'rem')

In [49]:
raw_df['stem-surface'].value_counts(dropna=False).head(10)

stem-surface
NaN    1980861
s       327610
y       255500
i       224346
t       147974
g        78080
k        73383
h        28283
f          512
w           49
Name: count, dtype: int64

In [50]:
stem_surface_lst = raw_df['stem-surface'].value_counts().head(7).index
raw_df['stem-surface'] = raw_df['stem-surface'].apply(lambda x: x if x in stem_surface_lst else 'rem')

In [51]:
view_categoricals_feature('stem-surface')

,p,e
stem-surface,,
g,99.610656,0.389344
h,99.635824,0.364176
i,63.854938,36.145062
k,69.795457,30.204543
rem,51.280346,48.719654
s,39.550685,60.449315
t,54.125725,45.874275
y,70.075930,29.924070


In [52]:
test_df['stem-surface'] = test_df['stem-surface'].apply(lambda x: x if x in stem_surface_lst else 'rem')

In [53]:
raw_df['stem-color'].value_counts().head(11)

stem-color
w    1196637
n    1003464
y     373971
g     132019
o     111541
e     103373
u      67017
p      54690
k      33676
r      22329
l       9994
Name: count, dtype: int64

In [54]:
stem_color_lst = raw_df['stem-color'].value_counts().head(10).index
raw_df['stem-color'] = raw_df['stem-color'].apply(lambda x: x if x in stem_color_lst else 'rem')

In [55]:
view_categoricals_feature('stem-color')

,p,e
stem-color,,
e,70.129531,29.870469
g,39.142093,60.857907
k,78.679178,21.320822
n,61.092775,38.907225
o,59.665952,40.334048
p,88.635948,11.364052
r,85.534507,14.465493
rem,34.781655,65.218345
u,62.545324,37.454676


In [56]:
test_df['stem-color'] = test_df['stem-color'].apply(lambda x: x if x in stem_color_lst else 'rem')

### Veil

In [57]:
raw_df['veil-type'].value_counts(dropna=False).head(10)

veil-type
NaN    2957493
u       159373
w           11
a            9
e            8
f            8
b            5
c            5
g            4
y            4
Name: count, dtype: int64

In [58]:
raw_df['veil-type'] = raw_df['veil-type'].apply(lambda x: x if x=='u' else 'rem')

In [59]:
view_categoricals_feature('veil-type')

,p,e
veil-type,,
rem,54.129333,45.870667
u,65.558156,34.441844


In [60]:
test_df['veil-type'] = test_df['veil-type'].apply(lambda x: x if x=='u' else 'rem')

In [61]:
raw_df['veil-color'].value_counts(dropna=False).head(10)

veil-color
NaN    2740947
w       279070
y        30473
n        30039
u        14026
k        13080
e         9169
g           30
p           23
r           14
Name: count, dtype: int64

In [62]:
veil_color_lst = ['w','y','n','u','k','e']
raw_df['veil-color'] = raw_df['veil-color'].apply(lambda x: x if x in veil_color_lst else 'rem')

In [63]:
view_categoricals_feature('veil-color')

,p,e
veil-color,,
e,99.901843,0.098157
k,99.915902,0.084098
n,99.627151,0.372849
rem,54.667818,45.332182
u,99.408242,0.591758
w,50.397750,49.602250
y,0.521773,99.478227


In [64]:
test_df['veil-color'] = test_df['veil-color'].apply(lambda x: x if x in veil_color_lst else 'rem')

### Ring

In [65]:
raw_df['has-ring'].value_counts(dropna=False).head(10)

has-ring
f      2368820
t       747982
NaN         24
r           16
h           13
c           11
l           11
s           11
p           11
g            8
Name: count, dtype: int64

In [66]:
has_ring_lst = ['f','t']
raw_df['has-ring'] = raw_df['has-ring'].apply(lambda x: x if x in has_ring_lst else 'f')

In [67]:
view_categoricals_feature('has-ring')

,p,e
has-ring,,
f,53.325485,46.674515
t,59.110380,40.889620


In [68]:
test_df['has-ring'] = test_df['has-ring'].apply(lambda x: x if x in has_ring_lst else 'f')

In [69]:
raw_df['ring-type'].value_counts(dropna=False).head(10)

ring-type
f      2477170
NaN     128880
e       120006
z       113780
l        73443
r        67909
p        67678
g        63687
m         3992
t           98
Name: count, dtype: int64

In [70]:
ring_type_lst = ['f','e','z','l','r','p','g']
raw_df['ring-type'] = raw_df['ring-type'].apply(lambda x: x if x in ring_type_lst else 'rem')

In [71]:
view_categoricals_feature('ring-type')

,p,e
ring-type,,
e,57.940436,42.059564
f,54.673842,45.326158
g,42.146749,57.853251
l,38.796073,61.203927
p,51.414049,48.585951
r,39.482248,60.517752
rem,38.428177,61.571823
z,99.615925,0.384075


In [72]:
test_df['ring-type'] = test_df['ring-type'].apply(lambda x: x if x in ring_type_lst else 'rem')

### Other

In [73]:
raw_df['spore-print-color'].value_counts(dropna=False).head(10)

spore-print-color
NaN    2849682
k       107310
p        68237
w        50173
n        22646
r         7975
u         7256
g         3492
y           36
s           21
Name: count, dtype: int64

In [74]:
spore_lst = ['k','p','w','n','r','u','g']
raw_df['spore-print-color'] = raw_df['spore-print-color'].apply(lambda x: x if x in spore_lst else 'rem')

In [75]:
view_categoricals_feature('spore-print-color')

,p,e
spore-print-color,,
g,2.835052,97.164948
k,82.536576,17.463424
n,99.726221,0.273779
p,71.231150,28.768850
r,99.724138,0.275862
rem,52.905971,47.094029
u,99.696803,0.303197
w,45.056106,54.943894


In [76]:
test_df['spore-print-color'] = test_df['spore-print-color'].apply(lambda x: x if x in spore_lst else 'rem')

In [77]:
raw_df['habitat'].value_counts(dropna=False).head(10)

habitat
d    2177573
g     454908
l     171892
m     150969
h     120137
w      18530
p      17180
u       5264
e         55
s         52
Name: count, dtype: int64

In [78]:
habitat_lst = ['d','g','l','m','h']
raw_df['habitat'] = raw_df['habitat'].apply(lambda x: x if x in habitat_lst else 'd')

In [79]:
view_categoricals_feature('habitat')

,p,e
habitat,,
d,52.683977,47.316023
g,67.460234,32.539766
h,66.914439,33.085561
l,39.492239,60.507761
m,53.761368,46.238632


In [80]:
test_df['habitat'] = test_df['habitat'].apply(lambda x: x if x in habitat_lst else 'd')

In [81]:
raw_df['season'].value_counts(dropna=False)

season
a    1543321
u    1153588
w     278189
s     141847
Name: count, dtype: int64

In [82]:
view_categoricals_feature('season')

,p,e
season,,
a,57.158621,42.841379
u,58.246098,41.753902
w,34.600937,65.399063
s,38.829866,61.170134


In [83]:
len(categorical_cols)

17

### Encode

In [84]:
from sklearn.preprocessing import LabelEncoder

In [85]:
for cat_col in categorical_cols:
    encoder = LabelEncoder()
    encoder.fit(raw_df[cat_col])
    raw_df[cat_col] = encoder.transform(raw_df[cat_col])
    test_df[cat_col] = encoder.transform(test_df[cat_col])

### Numeric cols

In [86]:
numeric_cols = numeric_cols + categorical_cols
len(numeric_cols)

20

In [87]:
raw_df[numeric_cols].isna().sum()

cap-diameter            4
stem-height             0
stem-width              0
cap-shape               0
cap-surface             0
cap-color               0
does-bruise-or-bleed    0
gill-attachment         0
gill-spacing            0
gill-color              0
stem-root               0
stem-surface            0
stem-color              0
veil-type               0
veil-color              0
has-ring                0
ring-type               0
spore-print-color       0
habitat                 0
season                  0
dtype: int64

In [88]:
avg_cap_diameter = raw_df['cap-diameter'].mean()
avg_cap_diameter

6.309848357732786

In [89]:
raw_df = raw_df.fillna({'cap-diameter': avg_cap_diameter})

In [90]:
test_df = test_df.fillna({'cap-diameter': avg_cap_diameter})

In [91]:
avg_stem_height = raw_df['stem-height'].mean()
test_df = test_df.fillna({'stem-height': avg_stem_height})

In [92]:
raw_df[numeric_cols].describe()

,cap-diameter,stem-height,stem-width,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
count,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06
mean,6.309848e+00,6.348333e+00,1.115379e+01,4.060597e+00,6.256271e+00,6.032857e+00,1.755196e-01,3.076800e+00,1.418834e+00,7.348291e+00,2.844256e+00,4.151050e+00,6.171596e+00,5.113116e-02,3.186041e+00,2.399728e-01,1.614152e+00,4.823069e+00,5.822156e-01,1.053464e+00
std,4.657928e+00,2.699755e+00,8.095477e+00,2.190271e+00,3.298277e+00,3.069461e+00,3.804110e-01,2.486081e+00,1.380239e+00,3.100741e+00,7.375861e-01,1.343073e+00,3.304713e+00,2.202652e-01,6.847690e-01,4.270666e-01,1.644103e+00,8.773727e-01,1.110150e+00,1.104258e+00
min,3.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.320000e+00,4.670000e+00,4.970000e+00,2.000000e+00,3.000000e+00,5.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,5.000000e+00,3.000000e+00,4.000000e+00,3.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,1.000000e+00,5.000000e+00,0.000000e+00,0.000000e+00
50%,5.750000e+00,5.880000e+00,9.650000e+00,5.000000e+00,7.000000e+00,5.000000e+00,0.000000e+00,3.000000e+00,1.000000e+00,7.000000e+00,3.000000e+00,4.000000e+00,9.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,1.000000e+00,5.000000e+00,0.000000e+00,1.000000e+00
75%,8.240000e+00,7.410000e+00,1.563000e+01,6.000000e+00,9.000000e+00,9.000000e+00,0.000000e+00,5.000000e+00,3.000000e+00,1.000000e+01,3.000000e+00,4.000000e+00,9.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,1.000000e+00,5.000000e+00,1.000000e+00,2.000000e+00
max,8.067000e+01,8.872000e+01,1.029000e+02,6.000000e+00,1.100000e+01,1.100000e+01,1.000000e+00,7.000000e+00,3.000000e+00,1.100000e+01,4.000000e+00,7.000000e+00,1.000000e+01,1.000000e+00,6.000000e+00,1.000000e+00,7.000000e+00,7.000000e+00,4.000000e+00,3.000000e+00


In [93]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(raw_df[numeric_cols])

MinMaxScaler()

In [94]:
raw_df[numeric_cols] = scaler.transform(raw_df[numeric_cols])
test_df[numeric_cols] = scaler.transform(test_df[numeric_cols])

In [95]:
raw_df[numeric_cols].describe()

,cap-diameter,stem-height,stem-width,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
count,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06,3.116945e+06
mean,7.787510e-02,7.155470e-02,1.083944e-01,6.767661e-01,5.687519e-01,5.484415e-01,1.755196e-01,4.395428e-01,4.729448e-01,6.680265e-01,7.110641e-01,5.930071e-01,6.171596e-01,5.113116e-02,5.310068e-01,2.399728e-01,2.305931e-01,6.890098e-01,1.455539e-01,3.511547e-01
std,5.776200e-02,3.043006e-02,7.867325e-02,3.650452e-01,2.998433e-01,2.790419e-01,3.804110e-01,3.551544e-01,4.600798e-01,2.818855e-01,1.843965e-01,1.918676e-01,3.304713e-01,2.202652e-01,1.141282e-01,4.270666e-01,2.348719e-01,1.253390e-01,2.775374e-01,3.680859e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.079861e-02,5.263751e-02,4.829932e-02,3.333333e-01,2.727273e-01,4.545455e-01,0.000000e+00,1.428571e-01,0.000000e+00,4.545455e-01,7.500000e-01,5.714286e-01,3.000000e-01,0.000000e+00,5.000000e-01,0.000000e+00,1.428571e-01,7.142857e-01,0.000000e+00,0.000000e+00
50%,7.093254e-02,6.627592e-02,9.378037e-02,8.333333e-01,6.363636e-01,4.545455e-01,0.000000e+00,4.285714e-01,3.333333e-01,6.363636e-01,7.500000e-01,5.714286e-01,9.000000e-01,0.000000e+00,5.000000e-01,0.000000e+00,1.428571e-01,7.142857e-01,0.000000e+00,3.333333e-01
75%,1.018105e-01,8.352119e-02,1.518950e-01,1.000000e+00,8.181818e-01,8.181818e-01,0.000000e+00,7.142857e-01,1.000000e+00,9.090909e-01,7.500000e-01,5.714286e-01,9.000000e-01,0.000000e+00,5.000000e-01,0.000000e+00,1.428571e-01,7.142857e-01,2.500000e-01,6.666667e-01
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [96]:
test_df[numeric_cols].describe()

,cap-diameter,stem-height,stem-width,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
count,2.077964e+06,2.077964e+06,2.077964e+06,2.077964e+06,2.077964e+06,2.077964e+06,2.077964e+06,2.077964e+06,2.077964e+06,2.077964e+06,2.077964e+06,2.077964e+06,2.077964e+06,2.077964e+06,2.077964e+06,2.077964e+06,2.077964e+06,2.077964e+06,2.077964e+06,2.077964e+06
mean,7.782977e-02,7.153414e-02,1.083418e-01,6.768382e-01,5.686309e-01,5.489980e-01,1.752807e-01,4.395138e-01,4.731508e-01,6.682512e-01,7.111223e-01,5.929752e-01,6.173561e-01,5.119097e-02,5.311614e-01,2.405042e-01,2.308218e-01,6.890412e-01,1.459084e-01,3.511604e-01
std,5.810335e-02,3.042129e-02,7.871896e-02,3.649727e-01,2.998027e-01,2.793189e-01,3.802071e-01,3.551620e-01,4.601191e-01,2.818084e-01,1.844336e-01,1.915896e-01,3.304735e-01,2.203871e-01,1.144538e-01,4.273898e-01,2.351206e-01,1.253816e-01,2.778578e-01,3.681928e-01
min,-3.720238e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.067460e-02,5.263751e-02,4.829932e-02,3.333333e-01,2.727273e-01,4.545455e-01,0.000000e+00,1.428571e-01,0.000000e+00,4.545455e-01,7.500000e-01,5.714286e-01,3.000000e-01,0.000000e+00,5.000000e-01,0.000000e+00,1.428571e-01,7.142857e-01,0.000000e+00,0.000000e+00
50%,7.080853e-02,6.627592e-02,9.368319e-02,8.333333e-01,6.363636e-01,4.545455e-01,0.000000e+00,4.285714e-01,3.333333e-01,6.363636e-01,7.500000e-01,5.714286e-01,9.000000e-01,0.000000e+00,5.000000e-01,0.000000e+00,1.428571e-01,7.142857e-01,0.000000e+00,3.333333e-01
75%,1.016865e-01,8.352119e-02,1.517979e-01,1.000000e+00,8.181818e-01,8.181818e-01,0.000000e+00,7.142857e-01,1.000000e+00,9.090909e-01,7.500000e-01,5.714286e-01,9.000000e-01,0.000000e+00,5.000000e-01,0.000000e+00,1.428571e-01,7.142857e-01,2.500000e-01,6.666667e-01
max,7.526910e+00,6.457394e-01,1.000097e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [97]:
input_cols = numeric_cols
target_col = 'class'

In [103]:
inputs = raw_df[input_cols]
targets = raw_df[target_col].apply(lambda x: 1 if x=='p' else 0)
test_inputs = test_df[input_cols]

In [99]:
from sklearn.model_selection import train_test_split

In [104]:
train_inputs, val_inputs, train_targets, val_targets = train_test_split(inputs, targets,
                                                                        test_size=0.25, random_state=42)

## Model

### Random Forest

In [102]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [114]:
sample_df = raw_df[input_cols+[target_col]].sample(600000)

In [115]:
sample_train_inputs, sample_val_inputs, sample_train_targets, sample_val_targets = train_test_split(
    sample_df[input_cols],
    sample_df[target_col].apply(lambda x: 1 if x=='p' else 0),
    test_size=0.2,
    random_state=42
)

In [116]:
classifiers = {
    'RF': RandomForestClassifier(n_jobs=-1),
    'XGB': XGBClassifier(n_jobs=-1),
    'LGBM': LGBMClassifier(n_jobs=-1)
}
result = {'Classifier': [], 'Accuracy': []}

In [117]:
for clf_name, clf in classifiers.items():
    clf.fit(sample_train_inputs, sample_train_targets)
    acc = clf.score(sample_val_inputs, sample_val_targets)
    result['Classifier'].append(clf_name)
    result['Accuracy'].append(acc)

result_df = pd.DataFrame(result)
result_df

,Classifier,Accuracy
0,RF,0.992108
1,XGB,0.991558
2,LGBM,0.989417


In [118]:
def test_params(**params):
    model = RandomForestClassifier(**params)
    model.fit(sample_train_inputs, sample_train_targets)
    train_acc = model.score(sample_train_inputs, sample_train_targets)
    val_acc = model.score(sample_val_inputs, sample_val_targets)
    return train_acc, val_acc

In [119]:
for i in [70, 90, 110, 150, 200, 250, 300, 400]:
    train_acc, val_acc = test_params(n_jobs=-1, n_estimators=i)
    print("Test {}: train_acc = {:6f}, val_acc = {:6f}".format(i, train_acc, val_acc))

Test 70: train_acc = 0.999942, val_acc = 0.991992
Test 90: train_acc = 0.999979, val_acc = 0.992142
Test 110: train_acc = 0.999988, val_acc = 0.992292
Test 150: train_acc = 0.999998, val_acc = 0.992192
Test 200: train_acc = 1.000000, val_acc = 0.992133
Test 250: train_acc = 1.000000, val_acc = 0.992200
Test 300: train_acc = 1.000000, val_acc = 0.992142
Test 400: train_acc = 1.000000, val_acc = 0.992192


In [121]:
for i in range(33, 42):
    train_acc, val_acc = test_params(n_jobs=-1, n_estimators=110,
                                     max_depth=i)
    print("Test {}: train_acc = {:6f}, val_acc = {:6f}".format(i, train_acc, val_acc))

Test 33: train_acc = 0.999471, val_acc = 0.992117
Test 34: train_acc = 0.999640, val_acc = 0.992142
Test 35: train_acc = 0.999708, val_acc = 0.992217
Test 36: train_acc = 0.999804, val_acc = 0.992158
Test 37: train_acc = 0.999875, val_acc = 0.992100
Test 38: train_acc = 0.999952, val_acc = 0.992125
Test 39: train_acc = 0.999956, val_acc = 0.992175
Test 40: train_acc = 0.999967, val_acc = 0.992142
Test 41: train_acc = 0.999979, val_acc = 0.992150


In [122]:
for i in ["gini", "entropy", "log_loss"]:
    train_acc, val_acc = test_params(n_jobs=-1, n_estimators=110,
                                     max_depth=35, criterion=i)
    print("Test {}: train_acc = {:6f}, val_acc = {:6f}".format(i, train_acc, val_acc))

Test gini: train_acc = 0.999692, val_acc = 0.992175
Test entropy: train_acc = 0.999865, val_acc = 0.992225
Test log_loss: train_acc = 0.999823, val_acc = 0.992242


In [123]:
for i in ["sqrt", "log2", None, 6, 9, 13, 17]:
    train_acc, val_acc = test_params(n_jobs=-1, n_estimators=110,
                                     max_depth=35, criterion='log_loss',
                                     max_features=i)
    print("Test {}: train_acc = {:6f}, val_acc = {:6f}".format(i, train_acc, val_acc))

Test sqrt: train_acc = 0.999867, val_acc = 0.992200
Test log2: train_acc = 0.999838, val_acc = 0.992108
Test None: train_acc = 0.999921, val_acc = 0.990567
Test 6: train_acc = 0.999879, val_acc = 0.991908
Test 9: train_acc = 0.999912, val_acc = 0.991825
Test 13: train_acc = 0.999898, val_acc = 0.991567
Test 17: train_acc = 0.999929, val_acc = 0.991117


In [124]:
for i in [None, 0, 17, 42, 123, 1212]:
    train_acc, val_acc = test_params(n_jobs=-1, n_estimators=110,
                                     max_depth=35, criterion='log_loss',
                                     random_state=i)
    print("Test {}: train_acc = {:6f}, val_acc = {:6f}".format(i, train_acc, val_acc))

Test None: train_acc = 0.999892, val_acc = 0.992192
Test 0: train_acc = 0.999865, val_acc = 0.992183
Test 17: train_acc = 0.999862, val_acc = 0.992258
Test 42: train_acc = 0.999858, val_acc = 0.992167
Test 123: train_acc = 0.999842, val_acc = 0.992167
Test 1212: train_acc = 0.999902, val_acc = 0.992275


In [127]:
for i in ['balanced', 'balanced_subsample', None, {0:1,1:1}, {0:1,1:1.2}, {0:1,1:1.5}, {0:1,1:2}]:
    train_acc, val_acc = test_params(n_jobs=-1, n_estimators=110,
                                     max_depth=35, criterion='log_loss',
                                     random_state=17, class_weight=i)
    print("Test {}: train_acc = {:6f}, val_acc = {:6f}".format(i, train_acc, val_acc))

Test balanced: train_acc = 0.999858, val_acc = 0.992225
Test balanced_subsample: train_acc = 0.999825, val_acc = 0.992108
Test None: train_acc = 0.999862, val_acc = 0.992258
Test {0: 1, 1: 1}: train_acc = 0.999862, val_acc = 0.992258
Test {0: 1, 1: 1.2}: train_acc = 0.999875, val_acc = 0.992167
Test {0: 1, 1: 1.5}: train_acc = 0.999854, val_acc = 0.992133
Test {0: 1, 1: 2}: train_acc = 0.999892, val_acc = 0.992167


In [128]:
rf_model = RandomForestClassifier(n_jobs=-1, random_state=17,
                                  n_estimators=110, max_depth=35,
                                  criterion='log_loss')

In [129]:
rf_model

RandomForestClassifier(criterion='log_loss', max_depth=35, n_estimators=110,
                       n_jobs=-1, random_state=17)

In [130]:
%%time

rf_model.fit(sample_train_inputs, sample_train_targets)

CPU times: total: 4min 30s
Wall time: 42.7 s


RandomForestClassifier(criterion='log_loss', max_depth=35, n_estimators=110,
                       n_jobs=-1, random_state=17)

In [131]:
rf_model.score(sample_val_inputs, sample_val_targets)

0.9922583333333334

In [132]:
rf_model.score(val_inputs, val_targets)

0.9929687630335828

In [133]:
preds = rf_model.predict(test_inputs)

In [137]:
preds = pd.Series(preds)
preds = preds.apply(lambda x: 'p' if x==1 else 'e')

In [138]:
sub_df['class'] = preds

In [139]:
sub_df.to_csv('poisonous_mushroom_data/sub1.csv', index=None)

In [140]:
%%time
rf_model.fit(train_inputs, train_targets)

CPU times: total: 36min 26s
Wall time: 6min 12s


RandomForestClassifier(criterion='log_loss', max_depth=35, n_estimators=110,
                       n_jobs=-1, random_state=17)

In [141]:
rf_model.score(val_inputs, val_targets)

0.9921346137311242

In [142]:
preds = rf_model.predict(test_inputs)

In [143]:
preds = pd.Series(preds)
preds = preds.apply(lambda x: 'p' if x==1 else 'e')

In [144]:
sub_df['class'] = preds
sub_df.to_csv('poisonous_mushroom_data/sub2.csv', index=None)

### XGBoost

In [145]:
from xgboost import XGBClassifier

In [148]:
def test_params(**params):
    model = XGBClassifier(**params)
    model.fit(sample_train_inputs, sample_train_targets)
    train_acc = model.score(sample_train_inputs, sample_train_targets)
    val_acc = model.score(sample_val_inputs, sample_val_targets)
    return train_acc, val_acc

In [149]:
test_params(n_jobs=-1)

(0.9918020833333333, 0.9915583333333333)

In [150]:
for i in [70,90, 120, 150, 250, 350]:
    train_acc, val_acc = test_params(n_jobs=-1, n_estimators=i)
    print("Test {}: train_acc = {:6f}, val_acc = {:6f}".format(i, train_acc, val_acc))

Test 70: train_acc = 0.990708, val_acc = 0.990725
Test 90: train_acc = 0.991602, val_acc = 0.991325
Test 120: train_acc = 0.992154, val_acc = 0.991658
Test 150: train_acc = 0.992542, val_acc = 0.991850
Test 250: train_acc = 0.993381, val_acc = 0.991917
Test 350: train_acc = 0.993960, val_acc = 0.991808


In [152]:
for i in range(4,9):
    train_acc, val_acc = test_params(n_jobs=-1, n_estimators=250,
                                     max_depth=i)
    print("Test {}: train_acc = {:6f}, val_acc = {:6f}".format(i, train_acc, val_acc))

Test 4: train_acc = 0.991325, val_acc = 0.991300
Test 5: train_acc = 0.992492, val_acc = 0.991633
Test 6: train_acc = 0.993381, val_acc = 0.991917
Test 7: train_acc = 0.994204, val_acc = 0.991792
Test 8: train_acc = 0.995085, val_acc = 0.991717


In [155]:
for i in [0.005,0.01,0.05,0.1,0.2,0.3,0.4]:
    train_acc, val_acc = test_params(n_jobs=-1, n_estimators=250,
                                     learning_rate=i)
    print("Test {}: train_acc = {:6f}, val_acc = {:6f}".format(i, train_acc, val_acc))

Test 0.005: train_acc = 0.919094, val_acc = 0.919092
Test 0.01: train_acc = 0.961629, val_acc = 0.961417
Test 0.05: train_acc = 0.987758, val_acc = 0.988058
Test 0.1: train_acc = 0.991242, val_acc = 0.990933
Test 0.2: train_acc = 0.992738, val_acc = 0.991975
Test 0.3: train_acc = 0.993381, val_acc = 0.991917
Test 0.4: train_acc = 0.993781, val_acc = 0.991800


In [158]:
for i in [0, 0.1, 0.2, 0.4, 0.7, 1, 1.2, 1.5, 2]:
    train_acc, val_acc = test_params(n_jobs=-1, n_estimators=250,
                                     learning_rate=0.2, reg_alpha=i)
    print("Test {}: train_acc = {:6f}, val_acc = {:6f}".format(i, train_acc, val_acc))

Test 0: train_acc = 0.992738, val_acc = 0.991975
Test 0.1: train_acc = 0.992706, val_acc = 0.991758
Test 0.2: train_acc = 0.992731, val_acc = 0.991750
Test 0.4: train_acc = 0.992663, val_acc = 0.991967
Test 0.7: train_acc = 0.992715, val_acc = 0.991925
Test 1: train_acc = 0.992752, val_acc = 0.992033
Test 1.2: train_acc = 0.992767, val_acc = 0.992058
Test 1.5: train_acc = 0.992687, val_acc = 0.991817
Test 2: train_acc = 0.992644, val_acc = 0.991825


In [159]:
for i in [0, 0.1, 0.2, 0.4, 0.7, 1, 1.1, 1.2, 1.5, 2]:
    train_acc, val_acc = test_params(n_jobs=-1, n_estimators=250,
                                     learning_rate=0.2, reg_alpha=1.2,
                                     reg_lambda=i)
    print("Test {}: train_acc = {:6f}, val_acc = {:6f}".format(i, train_acc, val_acc))

Test 0: train_acc = 0.992827, val_acc = 0.991892
Test 0.1: train_acc = 0.992750, val_acc = 0.991908
Test 0.2: train_acc = 0.992848, val_acc = 0.991783
Test 0.4: train_acc = 0.992783, val_acc = 0.991933
Test 0.7: train_acc = 0.992673, val_acc = 0.991842
Test 1: train_acc = 0.992767, val_acc = 0.992058
Test 1.1: train_acc = 0.992708, val_acc = 0.991842
Test 1.2: train_acc = 0.992610, val_acc = 0.991742
Test 1.5: train_acc = 0.992650, val_acc = 0.992000
Test 2: train_acc = 0.992687, val_acc = 0.991933


In [162]:
xgb_model = XGBClassifier(n_jobs=-1, n_estimators=250,
                          learning_rate=0.2, reg_alpha=1.2)

In [163]:
%%time
xgb_model.fit(sample_train_inputs, sample_train_targets)

CPU times: total: 30 s
Wall time: 6.8 s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=250, n_jobs=-1,
              num_parallel_tree=None, random_state=None, ...)

In [164]:
xgb_model.score(sample_val_inputs, sample_val_targets)

0.9920583333333334

In [165]:
xgb_model.score(val_inputs, val_targets)

0.9916944395607498

In [166]:
preds = xgb_model.predict(test_inputs)

In [167]:
preds = pd.Series(preds)
preds = preds.apply(lambda x: 'p' if x==1 else 'e')
preds

0          e
1          p
2          p
3          p
4          e
          ..
2077959    p
2077960    p
2077961    p
2077962    e
2077963    e
Length: 2077964, dtype: object

In [168]:
sub_df['class'] = preds
sub_df.to_csv('poisonous_mushroom_data/sub3.csv', index=None)

In [169]:
%%time
xgb_model.fit(train_inputs, train_targets)

CPU times: total: 3min 51s
Wall time: 52.5 s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=250, n_jobs=-1,
              num_parallel_tree=None, random_state=None, ...)

In [170]:
xgb_model.score(val_inputs, val_targets)

0.9918150703829516

In [171]:
preds = xgb_model.predict(test_inputs)
preds = pd.Series(preds)
preds = preds.apply(lambda x: 'p' if x==1 else 'e')
preds

0          e
1          p
2          p
3          p
4          e
          ..
2077959    p
2077960    p
2077961    p
2077962    e
2077963    e
Length: 2077964, dtype: object

In [172]:
sub_df['class'] = preds
sub_df.to_csv('poisonous_mushroom_data/sub4.csv', index=None)